<a href="https://colab.research.google.com/github/Savvythelegend/MLE/blob/main/projects/Time-Series%20Data%20Engineering/hf_data_handling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Reproducing the dataset
objective : the objective is to simulate the behaviour of high-frequency time-series data.

(which is data collected at regular intervals, often at high frequencies (like every 5ms).)

In [5]:
import pandas as pd
import numpy as np

In [6]:
# Create metadata lookup (ItemId -> ItemName, Comment, Unit)
meta = pd.DataFrame({
    'ItemId': [35827, 35828],
    'ItemName': ['L_R1SDW_SPDREF', 'L_R1SDD_SPDREF'],
    'Comment': ['R1 Screw Down WS Speed Reference (mps)', 'R1 Screw Down DS Speed Reference (mps)'],
    'Unit': ['mps', 'mps']
}).set_index('ItemName')
meta.head()

,ItemId,Comment,Unit
ItemName,,,
L_R1SDW_SPDREF,35827,R1 Screw Down WS Speed Reference (mps),mps
L_R1SDD_SPDREF,35828,R1 Screw Down DS Speed Reference (mps),mps


In [7]:
# Build a high-frequency timestamp series (5 ms) for 10 seconds -> 2000 samples
start = pd.to_datetime("2021-05-08 21:30:39")
timestamps = pd.date_range(start=start, periods=2000, freq='5ms')

In [8]:
timestamps.shape

(2000,)

In [9]:
# Create synthetic readings with a known max for each tag
np.random.seed(0)
vals_a = np.random.random(size=2000) * 30         # tag A values
vals_b = np.random.random(size=2000) * 40         # tag B values
# Insert a known max to test correctness
vals_a[123] = 99.123456
vals_b[987] = 88.654321

# Build input wide-style dataframe (like the CSV you'd get)
df_wide = pd.DataFrame({
    'timestamp': timestamps,
    'L_R1SDW_SPDREF': vals_a,
    'L_R1SDD_SPDREF': vals_b
})

In [10]:
df_wide

,timestamp,L_R1SDW_SPDREF,L_R1SDD_SPDREF
0,2021-05-08 21:30:39.000,16.464405,32.460739
1,2021-05-08 21:30:39.005,21.455681,19.043359
2,2021-05-08 21:30:39.010,18.082901,20.926240
3,2021-05-08 21:30:39.015,16.346495,10.020823
4,2021-05-08 21:30:39.020,12.709644,24.201721
...,...,...,...
1995,2021-05-08 21:30:48.975,23.738447,13.985575
1996,2021-05-08 21:30:48.980,21.635943,17.394945
1997,2021-05-08 21:30:48.985,14.403234,33.200118
1998,2021-05-08 21:30:48.990,19.315921,37.312247


In [11]:
# Melt (wide -> long)
df_long = df_wide.melt(id_vars=['timestamp'], var_name='tag_name', value_name='tag_value')
df_long.head(10)

,timestamp,tag_name,tag_value
0,2021-05-08 21:30:39.000,L_R1SDW_SPDREF,16.464405
1,2021-05-08 21:30:39.005,L_R1SDW_SPDREF,21.455681
2,2021-05-08 21:30:39.010,L_R1SDW_SPDREF,18.082901
3,2021-05-08 21:30:39.015,L_R1SDW_SPDREF,16.346495
4,2021-05-08 21:30:39.020,L_R1SDW_SPDREF,12.709644
5,2021-05-08 21:30:39.025,L_R1SDW_SPDREF,19.376823
6,2021-05-08 21:30:39.030,L_R1SDW_SPDREF,13.127616
7,2021-05-08 21:30:39.035,L_R1SDW_SPDREF,26.753190
8,2021-05-08 21:30:39.040,L_R1SDW_SPDREF,28.909883
9,2021-05-08 21:30:39.045,L_R1SDW_SPDREF,11.503246


In [12]:
# adding extra info
df_long['tag__id']   = df_long['tag_name'].map(meta['ItemId'])
df_long['tag__desc'] = df_long['tag_name'].map(meta['Comment'])
df_long['tag__unit'] = df_long['tag_name'].map(meta['Unit'])
df_long.head()

,timestamp,tag_name,tag_value,tag__id,tag__desc,tag__unit
0,2021-05-08 21:30:39.000,L_R1SDW_SPDREF,16.464405,35827,R1 Screw Down WS Speed Reference (mps),mps
1,2021-05-08 21:30:39.005,L_R1SDW_SPDREF,21.455681,35827,R1 Screw Down WS Speed Reference (mps),mps
2,2021-05-08 21:30:39.010,L_R1SDW_SPDREF,18.082901,35827,R1 Screw Down WS Speed Reference (mps),mps
3,2021-05-08 21:30:39.015,L_R1SDW_SPDREF,16.346495,35827,R1 Screw Down WS Speed Reference (mps),mps
4,2021-05-08 21:30:39.020,L_R1SDW_SPDREF,12.709644,35827,R1 Screw Down WS Speed Reference (mps),mps


In [13]:
# Aggregate to 10-second buckets using MAX
df_long['time_bucket'] = df_long['timestamp'].dt.floor('10S')

""".dt.floor('10S') rounds each timestamp down to the nearest 10-second boundary.
Example: 21:30:39.123 → 21:30:30
Example: 21:30:48.456 → 21:30:40

This creates “time buckets” so we can aggregate all 5 ms samples into 1 value per 10 seconds."""
agg = (df_long
       .groupby(['time_bucket', 'tag__id', 'tag_name', 'tag__desc', 'tag__unit'], as_index=False)
       .agg(tag__value=('tag_value', 'max')))
agg.head()

/tmp/ipython-input-1598080221.py:2: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  df_long['time_bucket'] = df_long['timestamp'].dt.floor('10S')


,time_bucket,tag__id,tag_name,tag__desc,tag__unit,tag__value
0,2021-05-08 21:30:30,35827,L_R1SDW_SPDREF,R1 Screw Down WS Speed Reference (mps),mps,99.123456
1,2021-05-08 21:30:30,35828,L_R1SDD_SPDREF,R1 Screw Down DS Speed Reference (mps),mps,39.581003
2,2021-05-08 21:30:40,35827,L_R1SDW_SPDREF,R1 Screw Down WS Speed Reference (mps),mps,29.994257
3,2021-05-08 21:30:40,35828,L_R1SDD_SPDREF,R1 Screw Down DS Speed Reference (mps),mps,88.654321


In [14]:
# Format event_timestamp as requested: dd/mm/YYYY HH:MM:SS
agg['event_timestamp'] = agg['time_bucket'].dt.strftime('%d/%m/%Y %H:%M:%S')

In [15]:
# Rearrange columns to match requested output order
out = agg[['event_timestamp','tag__id','tag_name','tag__desc','tag__value','tag__unit']]

print(out)
# Quick assertions to validate the known maxima:
assert out.loc[out['tag__id']==35827, 'tag__value'].max() == 99.123456
assert out.loc[out['tag__id']==35828, 'tag__value'].max() == 88.654321
print("Minimal test passed: MAX aggregation correct.")

       event_timestamp  tag__id        tag_name  \
0  08/05/2021 21:30:30    35827  L_R1SDW_SPDREF   
1  08/05/2021 21:30:30    35828  L_R1SDD_SPDREF   
2  08/05/2021 21:30:40    35827  L_R1SDW_SPDREF   
3  08/05/2021 21:30:40    35828  L_R1SDD_SPDREF   

                                tag__desc  tag__value tag__unit  
0  R1 Screw Down WS Speed Reference (mps)   99.123456       mps  
1  R1 Screw Down DS Speed Reference (mps)   39.581003       mps  
2  R1 Screw Down WS Speed Reference (mps)   29.994257       mps  
3  R1 Screw Down DS Speed Reference (mps)   88.654321       mps  
Minimal test passed: MAX aggregation correct.


In [16]:
out

,event_timestamp,tag__id,tag_name,tag__desc,tag__value,tag__unit
0,08/05/2021 21:30:30,35827,L_R1SDW_SPDREF,R1 Screw Down WS Speed Reference (mps),99.123456,mps
1,08/05/2021 21:30:30,35828,L_R1SDD_SPDREF,R1 Screw Down DS Speed Reference (mps),39.581003,mps
2,08/05/2021 21:30:40,35827,L_R1SDW_SPDREF,R1 Screw Down WS Speed Reference (mps),29.994257,mps
3,08/05/2021 21:30:40,35828,L_R1SDD_SPDREF,R1 Screw Down DS Speed Reference (mps),88.654321,mps


In [17]:
# show groups by tag_id
out.groupby('tag__id').agg({'tag__value': 'max'})

,tag__value
tag__id,
35827,99.123456
35828,88.654321


In [18]:
out.shape

(4, 6)

In [19]:
out.to_csv('output.csv', index=False)